In [ ]:
from sklearn import svm
from sklearn import datasets
import joblib
import pickle

In [ ]:
clf = svm.SVC()
iris = datasets.load_iris()
X, y = iris.data, iris.target
clf.fit(X,y)

SVC()

In [ ]:
#joblib
joblib.dump(clf, 'SVM.model')
clf2 = joblib.load('SVM.model')

print("gt: ", y[0:1])
print("predict: ", clf2.predict(X[0:1]))

gt:  [0]
predict:  [0]


In [ ]:
#pickle
with open('svm.pickle', 'wb') as f:
  pickle.dump(clf, f)
with open('svm.pickle', 'rb') as f:
  clf2 = pickle.load(f)

  print("gt: ", y[0:1])
  print("predict: ", clf2.predict(X[0:1]))

gt:  [0]
predict:  [0]


Resnet34

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torchvision.models import resnet34
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, TensorDataset

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# Add an extra dimension to represent batch
X_train = X_train.unsqueeze(1)
X_test = X_test.unsqueeze(1)

# Create a TensorDataset for train and test data
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Create data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the ResNet-34 classifier model
class ResNetClassifier(nn.Module):
    def __init__(self, num_classes):
        super(ResNetClassifier, self).__init__()
        self.resnet = resnet34(pretrained=True)
        self.resnet.conv1 = nn.Conv2d(64, 1, kernel_size=7, stride=2, padding=3, bias=False)
        self.resnet.fc = nn.Linear(512, num_classes)

    def forward(self, x):
      
        print(x.shape)
        return self.resnet(x)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set the training parameters
num_classes = len(iris.target_names)
num_epochs = 10
learning_rate = 0.001

# Create the ResNet-34 classifier model
model = ResNetClassifier(num_classes).to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)

    train_loss /= len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}")

# Evaluation
model.eval()
y_pred = []
y_true = []

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)

        y_pred.extend(predicted.cpu().tolist())
        y_true.extend(targets.cpu().tolist())

accuracy = accuracy_score(y_true, y_pred)
print("Test Accuracy:", accuracy)

torch.save(model.state_dict(), 'resnet.pth')
print("Model weights saved successfully.")

(150, 4)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


torch.Size([64, 1, 4])


ValueError: ignored

In [ ]:
import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical
import torch.nn.functional as F

import torch.optim as optim
from torchvision.models import resnet34
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Dataset

In [ ]:
# Define the ResNet-34 classifier model
class ResNetClassifier(nn.Module):
    def __init__(self, num_classes):
        super(ResNetClassifier, self).__init__()
        self.resnet = resnet34(pretrained=True)
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.resnet.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        return self.resnet(x)
class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_dim,50)
        self.layer2 = nn.Linear(50, 20)
        self.layer3 = nn.Linear(20, 3)
        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.softmax(self.layer3(x)) # To check with the loss function
        return x

In [ ]:
# features, labels = load_iris(return_X_y=True)
features_train,features_test, labels_train, labels_test = train_test_split(X, y, random_state=42, shuffle=True)

In [ ]:
# Training
model = Model(features_train.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()
epochs = 100

In [ ]:
x_train, y_train = Variable(torch.from_numpy(features_train)).float(), Variable(torch.from_numpy(labels_train)).long()
for epoch in range(1, epochs+1):
    print ("Epoch #",epoch)
    y_pred = model(x_train)
    loss = loss_fn(y_pred, y_train)
    print(loss.item())
    
    # Zero gradients
    optimizer.zero_grad()
    loss.backward() # Gradients
    optimizer.step() # Update
torch.save(model.state_dict(), 'resnet.pth')
print("Model weights saved successfully.")

In [ ]:
class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_dim,50)
        self.layer2 = nn.Linear(50, 20)
        self.layer3 = nn.Linear(20, 3)
        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.softmax(self.layer3(x)) # To check with the loss function
        return x

In [ ]:
import joblib
import torch
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
from torchvision.datasets import DatasetFolder
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Load the SVM model weights
svm_model = SVC()
svm_model = joblib.load("svm.model")

# Load the MLP model weights
mlp_model = Model()
mlp_model.load_state_dict(torch.load("mlp.pth"))

# Make predictions
svm_predictions = svm_model.predict(X_test)
mlp_predictions = mlp_model.predict(X_test)

# Ensemble the predictions
ensemble_predictions = []
for svm_pred, mlp_pred in zip(svm_predictions, mlp_predictions):
    # Voting: choose the majority vote
    if svm_pred == mlp_pred:
        ensemble_predictions.append(svm_pred)

# Evaluate the ensemble accuracy
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
print("Ensemble Accuracy:", ensemble_accuracy)
